In [65]:
%load_ext autoreload
%autoreload 2

import mycoco

mycoco.setmode('train')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading annotations into memory...
Done (t=13.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.91s)
creating index...
index created!


In [66]:
all_ids = mycoco.query([['']])

In [67]:
import pickle

with open('./data/tokenizer10000.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
    
tokenizer

In [73]:
from keras.models import load_model

encoder = load_model('./models/encoder.model.hdf5')
encoder.compile()

/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


TypeError: compile() missing 1 required positional argument: 'optimizer'

In [69]:
cap_examples = mycoco.iter_captions_examples(all_ids, tokenizer, encoder)

In [70]:
next(cap_examples)[1]

/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)],
 array([[0.46396238, 0.23454374, 0.        , ..., 0.08909912, 0.31195068,
         0.29884964]], dtype=float32)]

In [71]:
from keras.models import Model
from keras.layers import Dense, LSTM, Embedding, Input

input_length = 5
embed_size = 50
vocab_size = 10000
img_vec_size = 5000

inputs = Input(shape=(input_length,))
embed = Embedding(vocab_size, embed_size, input_length=input_length)(inputs)
lstm = LSTM(50, dropout=0.1)(embed)
# Word prediction softmax
word_pred = Dense(vocab_size, activation='softmax', name='word_prediction')(lstm)
image_vec_preds = Dense(img_vec_size, activation = 'sigmoid', name='image_vec_prediction')(lstm)

# This creates a model that includes
# the Input layer and two Dense layers outputs
model = Model(inputs=inputs, outputs=[word_pred, image_vec_preds])

model.compile(optimizer='adam',
        loss={
            'word_prediction': 'categorical_crossentropy',
            'image_vec_prediction': 'binary_crossentropy'
        },
        metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 5, 50)        500000      input_6[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, 50)           20200       embedding_6[0][0]                
__________________________________________________________________________________________________
word_prediction (Dense)         (None, 10000)        510000      lstm_6[0][0]                     
__________________________________________________________________________________________________
image_vec_

In [72]:
model.fit_generator(cap_examples, steps_per_epoch=100, epochs=30)

Epoch 1/30
 41/100 [===========>..................] - ETA: 2s - loss: 9.8874 - word_prediction_loss: 9.2016 - image_vec_prediction_loss: 0.6858 - word_prediction_acc: 0.0244 - image_vec_prediction_acc: 0.2743

/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


100/100 [==============================] - 3s 30ms/step - loss: 9.5729 - word_prediction_loss: 8.9611 - image_vec_prediction_loss: 0.6118 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.3154
Epoch 2/30
100/100 [==============================] - 2s 19ms/step - loss: 8.0058 - word_prediction_loss: 7.7042 - image_vec_prediction_loss: 0.3016 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.3550
Epoch 3/30
100/100 [==============================] - 2s 19ms/step - loss: 7.9948 - word_prediction_loss: 7.3949 - image_vec_prediction_loss: 0.5998 - word_prediction_acc: 0.0800 - image_vec_prediction_acc: 0.2466
Epoch 4/30
100/100 [==============================] - 2s 19ms/step - loss: 7.3477 - word_prediction_loss: 6.8338 - image_vec_prediction_loss: 0.5139 - word_prediction_acc: 0.0500 - image_vec_prediction_acc: 0.2932
Epoch 5/30
100/100 [==============================] - 2s 19ms/step - loss: 7.3283 - word_prediction_loss: 6.8762 - image_vec_prediction_loss: 0.4521 - word

In [16]:
import pickle

with open('./data/tokenizer10000_oov.pickle', 'rb')  as f:
    tokenizer = pickle.load(f)

In [22]:
word_lookup = {v: k for k, v in tokenizer.word_index.items() if v < 10000} 

In [25]:
tokenizer.word_index

{'UNK': 1,
 'a': 2,
 'on': 3,
 'of': 4,
 'the': 5,
 'in': 6,
 'with': 7,
 'and': 8,
 'is': 9,
 'man': 10,
 'to': 11,
 'sitting': 12,
 'an': 13,
 'two': 14,
 'at': 15,
 'standing': 16,
 'people': 17,
 'are': 18,
 'next': 19,
 'white': 20,
 'woman': 21,
 'street': 22,
 'table': 23,
 'that': 24,
 'holding': 25,
 'it': 26,
 'large': 27,
 'person': 28,
 'some': 29,
 'down': 30,
 'top': 31,
 'group': 32,
 'up': 33,
 'field': 34,
 'small': 35,
 'tennis': 36,
 'near': 37,
 'front': 38,
 'black': 39,
 'his': 40,
 'train': 41,
 'plate': 42,
 'room': 43,
 'riding': 44,
 'dog': 45,
 'red': 46,
 'cat': 47,
 'young': 48,
 'by': 49,
 'water': 50,
 'baseball': 51,
 'has': 52,
 'playing': 53,
 'while': 54,
 'walking': 55,
 'bathroom': 56,
 'sign': 57,
 'blue': 58,
 'kitchen': 59,
 'food': 60,
 'grass': 61,
 'bus': 62,
 'there': 63,
 'green': 64,
 'pizza': 65,
 'side': 66,
 'parked': 67,
 'building': 68,
 'other': 69,
 'bed': 70,
 'looking': 71,
 'snow': 72,
 'beach': 73,
 'ball': 74,
 'couple': 75,
 't

In [23]:
print(word_lookup[9999], word_lookup[10000])

KeyError: 10000

In [19]:
tokenizer.texts_to_sequences(["waterboard loosing"])

[[9999, 1]]

In [85]:
%load_ext autoreload
%autoreload 2

import mycoco
from keras.models import load_model
import csv
import numpy as np

mycoco.setmode('train')

encoder = load_model('./models/encoder.model.hdf5')

all_ids = mycoco.query([['']])

all_vectors_iter = mycoco.iter_vector_images(all_ids[0])

with open('./data/id_vector_rep.csv', 'w+') as f:
    writer = csv.writer(f)
    for i, img in all_vectors_iter:
        vec = encoder.predict(img)
        
        row = np.insert(vec, 0, i)
        writer.writerow(row)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading annotations into memory...
Done (t=15.61s)
creating index...
index created!
loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyboardInterrupt: 

(5001,)
524333.0


/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [56]:
#all_preds = encoder.predict_generator(all_vectors_iter, )
i, img = next(all_vectors_iter)

/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [57]:
encoder.predict(img)

array([[0.3912587 , 0.23456162, 0.78462696, ..., 0.17266646, 0.09202953,
        1.0648549 ]], dtype=float32)

In [1]:
from keras.models import load_model

encoder = load_model('./models/encoder.model.hdf5')

/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
autoencoder = load_model('./models/autoencoder.model.hdf5')


TensorShape([Dimension(None), Dimension(None)])

In [27]:
autoencoder.get_layer('encoded_flat').output_shape[1]
#encoder.get_output_at(0).get_shape().as_list()

5000

In [21]:
layer.get_output_at(0).get_shape().as_list()

[None, None]

In [13]:
%%time

import mycoco
from keras.models import load_model, Model
import csv
import numpy as np

mycoco.setmode('train')

autoencoder = load_model('./models/autoencoder.model.hdf5')
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoded_flat').output)

all_ids = mycoco.query([['']])

all_vectors_iter = mycoco.iter_vector_images(all_ids[0])

batch = 10000
ids = []
images = []
enc_images = []
j = 0
for i, img in all_vectors_iter:
    images.append(img[0])
        #vec = encoder.predict(img) #.round(6)
    ids.append(i)
    #images.append(vec)
    if j >= batch:
        new_enc = encoder.predict(np.array(images))
        images = []
        enc_images.append(new_enc)
        j = 0
    j += 1

loading annotations into memory...
Done (t=14.47s)
creating index...
index created!
loading annotations into memory...
Done (t=0.90s)
creating index...
index created!


/usr/lib64/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


CPU times: user 34min 44s, sys: 31min 46s, total: 1h 6min 30s
Wall time: 24min 23s


In [ ]:
len(all_ids[0]) / 10000

In [8]:
import numpy as np
np.concatenate([np.array([4,5,6]), np.array([1,2,3])])

array([4, 5, 6, 1, 2, 3])

In [3]:
import pandas as pd
#df = pd.read_csv('./data/enc_images.csv', index_col=0)
df = pd.read_csv('./data/enc_images.csv.tar.gz', compression='gzip', index_col=0)

In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
enc_images.csv,,,,,,,,,,,,,,,,,,,,,
262145.0,1.0595,0.4507,0.5821,0.7193,0.0,0.6350,1.1965,0.6008,0.8202,0.9691,...,1.8208,1.0169,0.9766,1.0252,0.7917,1.2739,0.2094,0.8217,0.6973,0.4758
262146.0,0.6273,0.6482,0.6832,1.3112,0.0,0.6248,0.4924,0.3818,0.5495,0.4603,...,1.4698,0.4269,0.5236,0.1788,0.2298,0.5344,0.1940,0.8114,1.4558,0.4643
524291.0,0.2797,0.9612,0.7817,0.8295,0.0,0.5273,1.1870,0.4809,0.3970,0.2874,...,0.9657,0.8883,0.4114,0.8186,0.8746,1.2960,0.3360,0.6255,0.8610,0.4234
262148.0,0.4020,0.9708,0.8064,1.9071,0.0,0.0526,0.0000,0.0276,0.2554,1.5287,...,0.5184,0.8203,0.8441,0.2980,0.4702,1.9338,0.0000,0.1856,0.0000,0.1650
524297.0,0.3835,0.2315,0.7018,0.2354,0.0,0.4862,1.6331,0.6903,0.3725,0.3528,...,2.1560,0.1075,0.5281,0.0189,0.0000,0.2833,0.1889,0.7394,2.0274,0.3901
9.0,0.8671,0.6170,0.9899,0.8297,0.0,0.0000,0.9112,0.2190,0.6768,0.6416,...,0.3532,0.5234,0.6801,0.6270,0.5566,0.8513,0.2601,0.8110,0.7696,0.7170
262159.0,0.2704,0.0000,0.4130,0.5158,0.0,0.5945,1.7814,1.1041,0.4438,0.3485,...,0.9003,0.4514,0.5015,0.3392,0.4468,0.8947,0.0504,0.8127,0.8814,0.6006
262161.0,0.4727,0.4511,0.5943,0.6275,0.0,0.7195,1.1842,0.3991,0.6323,0.2224,...,0.5935,0.6819,0.5428,0.3859,0.5982,1.1013,0.0902,0.9162,0.6217,0.6743
262162.0,0.4220,0.5900,0.7036,0.7070,0.0,0.6258,1.0572,0.4184,0.4008,0.4017,...,0.5792,0.6306,0.7236,0.5414,0.5914,1.1620,0.0932,0.6509,0.6426,0.3559


In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
x = np.random.random(200)

In [13]:
nbrs = NearestNeighbors(n_neighbors=3).fit(df)
# nbrs.kneighbors(x)

In [34]:
x = df.iloc[255]
for z in df.index[nbrs.kneighbors([x], return_distance=False)[0]]:
    print(z)

262550
12343
176617


In [17]:
x - df.iloc[20586]

5.5716547859338466